In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(f"device is {device}")

device is cuda:0


# Without AutoGrad

In [2]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(1000):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 494.78753662109375
199 1.6378469467163086
299 0.008630793541669846
399 0.00019382868777029216
499 3.6546451156027615e-05
599 1.5714915207354352e-05
699 9.290351044910494e-06
799 6.4721557464508805e-06
899 4.67301197204506e-06
999 3.6642975373979425e-06


# With AutoGrad

In [3]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(1000):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss)

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

99 tensor(203.1118, device='cuda:0', grad_fn=<SumBackward0>)
199 tensor(0.8218, device='cuda:0', grad_fn=<SumBackward0>)
299 tensor(0.0059, device='cuda:0', grad_fn=<SumBackward0>)
399 tensor(0.0002, device='cuda:0', grad_fn=<SumBackward0>)
499 tensor(3.8745e-05, device='cuda:0', grad_fn=<SumBackward0>)
599 tensor(1.6379e-05, device='cuda:0', grad_fn=<SumBackward0>)
699 tensor(9.6917e-06, device='cuda:0', grad_fn=<SumBackward0>)
799 tensor(6.7828e-06, device='cuda:0', grad_fn=<SumBackward0>)
899 tensor(4.9964e-06, device='cuda:0', grad_fn=<SumBackward0>)
999 tensor(3.8024e-06, device='cuda:0', grad_fn=<SumBackward0>)


# Pytorch package: nn

In [4]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device=device)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 tensor(2.2018, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(0.0370, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0013, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(6.6598e-05, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(4.0244e-06, device='cuda:0', grad_fn=<MseLossBackward>)


# Pytorch package: optim

In [5]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device=device)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 tensor(61.0469, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(1.8098, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0252, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(1.9263e-07, device='cuda:0', grad_fn=<MseLossBackward>)


# Pytorch package: nn.Module

In [7]:
dtype, N, D_in, H, D_out = torch.float, 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.relu    = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred

model = TwoLayerNet(D_in, H, D_out)
model.to(device=device)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 tensor(2.2798, device='cuda:0', grad_fn=<MseLossBackward>)
199 tensor(0.0364, device='cuda:0', grad_fn=<MseLossBackward>)
299 tensor(0.0011, device='cuda:0', grad_fn=<MseLossBackward>)
399 tensor(4.0086e-05, device='cuda:0', grad_fn=<MseLossBackward>)
499 tensor(1.7340e-06, device='cuda:0', grad_fn=<MseLossBackward>)
